# Chapter 16: Surrogate Optimization

In [3]:
import numpy as np
from scipy import stats

## Algorithm 16.1

In [4]:
prob_of_improvement = lambda y_min, mu, sigma: stats.norm(mu,sigma).cdf(y_min)

## Algorithm 16.2

In [7]:
def expected_improvement(y_min, mu, sigma):
    p_imp = prob_of_improvement(y_min, mu, sigma)
    p_ymin = stats.norm(mu, sigma).pdf(y_min)
    return (y_min - mu)*p_imp + sigma*p_ymin